# Bayesian Optimization

In [25]:
import torch
from torch.quasirandom import SobolEngine
from botorch.models import SingleTaskGP
from botorch.optim.fit import fit_gpytorch_mll_torch
from botorch.acquisition import ExpectedImprovement
from botorch.optim import optimize_acqf
from gpytorch.mlls import ExactMarginalLogLikelihood

#### Define the Objective Function

The objective function is the function you want to optimize. It takes hyperparameters as input and returns the model's cross-validation score as the output.



In [26]:
# Define the objective function
# Function to minimize: f(x) = (x - 2)^2
def objective_function(x):
    return (x - 2) ** 2

#### Set Up the Bayesian Optimizer

Define the bounds for the hyperparameters and initialize the Bayesian optimizer.



In [ ]:
# Perform Bayesian Optimization
def bayesian_optimization():
    # Step 1: Define the search space
    bounds = torch.tensor([[0.0], [4.0]])  # Search space: x in [0, 4]

    # Step 2: Generate initial data
    num_initial_points = 5
    sobol = SobolEngine(dimension=1, scramble=True)
    train_x = sobol.draw(num_initial_points) * (bounds[1] - bounds[0]) + bounds[0]  # [N, d] tensor
    train_y = objective_function(train_x)  # Compute objective values (shape: [N, 1])

    # Step 3: Fit a Gaussian Process (GP) model
    gp = SingleTaskGP(train_x, train_y)  # Ensure train_y is 2D with shape [N, 1]
    mll = ExactMarginalLogLikelihood(gp.likelihood, gp)
    fit_gpytorch_mll_torch(mll)

    # Step 4: Optimization loop
    num_iterations = 10
    for i in range(num_iterations):
        # Define the acquisition function
        ei = ExpectedImprovement(gp, best_f=train_y.min())

        # Optimize the acquisition function to propose the next point
        candidate, _ = optimize_acqf(
            acq_function=ei,
            bounds=bounds,
            q=1,  # Number of candidates to optimize for
            num_restarts=10,
            raw_samples=100,
        )

        # Evaluate the objective function at the candidate point
        new_x = candidate.detach()
        new_y = objective_function(new_x)  # Ensure new_y is 2D with shape [N, 1]

        # Update the training data
        train_x = torch.cat([train_x, new_x])  # Concatenate along rows
        train_y = torch.cat([train_y, new_y])  # Concatenate along rows

        # Refit the GP model
        gp = SingleTaskGP(train_x, train_y)
        mll = ExactMarginalLogLikelihood(gp.likelihood, gp)
        fit_gpytorch_mll_torch(mll)

        # Print progress
        print(f"Iteration {i + 1}: x = {new_x.item():.4f}, f(x) = {new_y.item():.4f}")

    # Step 5: Return the best result
    best_index = train_y.argmin()
    best_x = train_x[best_index].item()  # Convert tensor to scalar
    best_y = train_y[best_index].item()  # Convert tensor to scalar
    return best_x, best_y

#### Run the Optimization

In [28]:
# Run the Bayesian Optimization
best_x, best_y = bayesian_optimization()
print(f"Optimal x: {best_x}, Optimal f(x): {best_y}")

BotorchTensorDimensionError: Expected X and Y to have the same number of dimensions (got X with dimension 2 and Y with dimension 3).

#### Advantages of Bayesian Optimization
Efficiency: It finds good hyperparameters with fewer iterations compared to Grid Search or Random Search.  
Balance of Exploration and Exploitation: Bayesian optimization intelligently balances exploring new areas of the hyperparameter space and exploiting areas that have already shown promise.  
Works with Noisy Objectives: It performs well even when the objective function is noisy (e.g., stochastic models).  

#### When to Use Bayesian Optimization
When the hyperparameter space is large or contains continuous variables.  
When computational resources are limited, and you want to optimize efficiently.  
For machine learning models where training is expensive (e.g., deep learning).  